<a href="https://colab.research.google.com/github/vandarsar/Linear-regression/blob/main/Arcada_coursework_final_project_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [352]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
%matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score # MAE, RMSE, R2 score
from sklearn.model_selection import cross_val_score,KFold #to be used for cross validation
from sklearn.linear_model import Ridge, Lasso #for Ridge and Lasso Regression
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve    # for plotting a roc curve
from sklearn.metrics import roc_auc_score  # for calculating auc score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV   # importing Gridsearch for hyperparameter and model optimisation
from sklearn.model_selection import RandomizedSearchCV # randomised gridgsearch
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler #standardization scaler library
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # to import pipeline scaler
import plotly.graph_objects as go


In [353]:
df = pd.read_csv("https://people.arcada.fi/~parland/hjd5_8amp_Gt3/EURUSD1m.zip", compression='zip', parse_dates=["Date"])

In [354]:
df.head()    #checking the head

,Date,Timestamp,Open,High,Low,Close,Volume
0,2010-01-01,11:00:00,1.43327,1.43335,1.43319,1.43335,56.299999
1,2010-01-01,11:01:00,1.43333,1.43333,1.43318,1.43327,82.100001
2,2010-01-01,11:02:00,1.43328,1.43333,1.43319,1.43333,86.900000
3,2010-01-01,11:03:00,1.43325,1.43333,1.43319,1.43326,68.899999
4,2010-01-01,11:04:00,1.43326,1.43333,1.43319,1.43319,45.300000


In [355]:
df["Date"] = df.Date.astype(str)   # converting Date column to str for eventual concatenation

In [356]:
df.head()

,Date,Timestamp,Open,High,Low,Close,Volume
0,2010-01-01,11:00:00,1.43327,1.43335,1.43319,1.43335,56.299999
1,2010-01-01,11:01:00,1.43333,1.43333,1.43318,1.43327,82.100001
2,2010-01-01,11:02:00,1.43328,1.43333,1.43319,1.43333,86.900000
3,2010-01-01,11:03:00,1.43325,1.43333,1.43319,1.43326,68.899999
4,2010-01-01,11:04:00,1.43326,1.43333,1.43319,1.43319,45.300000


In [357]:
df.dtypes     # checking the data types of date and timestamp column to be sure they are now str

Date          object
Timestamp     object
Open         float64
High         float64
Low          float64
Close        float64
Volume       float64
dtype: object

In [358]:
# concantenate the 2 columns with white space in between so pandas can correctly recognise it

df["Date"] = df['Date'] + ' ' + df['Timestamp']   

In [359]:
df.head()

,Date,Timestamp,Open,High,Low,Close,Volume
0,2010-01-01 11:00:00,11:00:00,1.43327,1.43335,1.43319,1.43335,56.299999
1,2010-01-01 11:01:00,11:01:00,1.43333,1.43333,1.43318,1.43327,82.100001
2,2010-01-01 11:02:00,11:02:00,1.43328,1.43333,1.43319,1.43333,86.900000
3,2010-01-01 11:03:00,11:03:00,1.43325,1.43333,1.43319,1.43326,68.899999
4,2010-01-01 11:04:00,11:04:00,1.43326,1.43333,1.43319,1.43319,45.300000


In [360]:
df["Date"] = pd.to_datetime(df["Date"]) # convert date to datetime object

In [361]:
df.dtypes #checking the data type showed we now have datetime64[ns]

Date         datetime64[ns]
Timestamp            object
Open                float64
High                float64
Low                 float64
Close               float64
Volume              float64
dtype: object

In [362]:
df.columns   

Index(['Date', 'Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [363]:
df.columns = ['date', 'timestamp', 'open', 'high', 'low', 'close', 'volume'] # changed the column

In [364]:
df.head() # column changed

,date,timestamp,open,high,low,close,volume
0,2010-01-01 11:00:00,11:00:00,1.43327,1.43335,1.43319,1.43335,56.299999
1,2010-01-01 11:01:00,11:01:00,1.43333,1.43333,1.43318,1.43327,82.100001
2,2010-01-01 11:02:00,11:02:00,1.43328,1.43333,1.43319,1.43333,86.900000
3,2010-01-01 11:03:00,11:03:00,1.43325,1.43333,1.43319,1.43326,68.899999
4,2010-01-01 11:04:00,11:04:00,1.43326,1.43333,1.43319,1.43319,45.300000


In [365]:
df.drop(["timestamp"], axis=1) # dropping the timestamp column

,date,open,high,low,close,volume
0,2010-01-01 11:00:00,1.43327,1.43335,1.43319,1.43335,56.299999
1,2010-01-01 11:01:00,1.43333,1.43333,1.43318,1.43327,82.100001
2,2010-01-01 11:02:00,1.43328,1.43333,1.43319,1.43333,86.900000
3,2010-01-01 11:03:00,1.43325,1.43333,1.43319,1.43326,68.899999
4,2010-01-01 11:04:00,1.43326,1.43333,1.43319,1.43319,45.300000
...,...,...,...,...,...,...
3735009,2019-12-31 11:55:00,1.12266,1.12267,1.12250,1.12251,196.390000
3735010,2019-12-31 11:56:00,1.12250,1.12261,1.12247,1.12251,167.640000
3735011,2019-12-31 11:57:00,1.12254,1.12273,1.12254,1.12271,164.570002
3735012,2019-12-31 11:58:00,1.12271,1.12283,1.12262,1.12275,244.780001


In [366]:
df.set_index("date", inplace=True) # re-seting the index to date column

In [367]:
df.index # this shows the index object

DatetimeIndex(['2010-01-01 11:00:00', '2010-01-01 11:01:00',
               '2010-01-01 11:02:00', '2010-01-01 11:03:00',
               '2010-01-01 11:04:00', '2010-01-01 11:05:00',
               '2010-01-01 11:06:00', '2010-01-01 11:07:00',
               '2010-01-01 11:08:00', '2010-01-01 11:09:00',
               ...
               '2019-12-31 11:50:00', '2019-12-31 11:51:00',
               '2019-12-31 11:52:00', '2019-12-31 11:53:00',
               '2019-12-31 11:54:00', '2019-12-31 11:55:00',
               '2019-12-31 11:56:00', '2019-12-31 11:57:00',
               '2019-12-31 11:58:00', '2019-12-31 11:59:00'],
              dtype='datetime64[ns]', name='date', length=3735014, freq=None)

In [368]:
df.head(2)

,timestamp,open,high,low,close,volume
date,,,,,,
2010-01-01 11:00:00,11:00:00,1.43327,1.43335,1.43319,1.43335,56.299999
2010-01-01 11:01:00,11:01:00,1.43333,1.43333,1.43318,1.43327,82.100001


In [369]:
df.head()

,timestamp,open,high,low,close,volume
date,,,,,,
2010-01-01 11:00:00,11:00:00,1.43327,1.43335,1.43319,1.43335,56.299999
2010-01-01 11:01:00,11:01:00,1.43333,1.43333,1.43318,1.43327,82.100001
2010-01-01 11:02:00,11:02:00,1.43328,1.43333,1.43319,1.43333,86.900000
2010-01-01 11:03:00,11:03:00,1.43325,1.43333,1.43319,1.43326,68.899999
2010-01-01 11:04:00,11:04:00,1.43326,1.43333,1.43319,1.43319,45.300000


In [370]:
# Subsample data to one day timesteps, be sure to also get data from weekends.

In [371]:
df = df.resample("D").mean()  # subsampling the DataFrame to 1-Day interval by taking the mean of data point within 24hrs

In [372]:
df.tail() # checked the df's tail here to get better sense of the data distrubution

,open,high,low,close,volume
date,,,,,
2019-12-27,1.114447,1.114498,1.114401,1.114452,94.375684
2019-12-28,NaN,NaN,NaN,NaN,NaN
2019-12-29,1.117864,1.117915,1.117801,1.117858,27.491448
2019-12-30,1.119915,1.119964,1.119867,1.119915,90.878301
2019-12-31,1.120863,1.120901,1.120828,1.120866,61.131195


In [373]:
# Create a Label column for your forecast, by shifting the Close value 1 step. You will predict one day ahead.


In [374]:
df["label"] = df.close.shift(1)   # creating a new column from close by shifting the close by one day

In [375]:
df.fillna(0, inplace=True)
df.head(20)  # head shows the new column has been created. we now know there are missing values

,open,high,low,close,volume,label
date,,,,,,
2010-01-01,1.433194,1.433288,1.433089,1.433178,60.243939,0.000000
2010-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,1.433178
2010-01-03,1.431891,1.432094,1.431711,1.431913,25.013333,0.000000
2010-01-04,1.436431,1.436693,1.436159,1.436451,55.569028,1.431913
2010-01-05,1.440870,1.441108,1.440588,1.440881,55.477153,1.436451
2010-01-06,1.436875,1.437116,1.436592,1.436898,56.230417,1.440881
2010-01-07,1.435212,1.435446,1.434926,1.435225,56.116320,1.436898
2010-01-08,1.433802,1.434065,1.433514,1.433824,57.691212,1.435225
2010-01-09,0.000000,0.000000,0.000000,0.000000,0.000000,1.433824


In [376]:
df.describe()

,open,high,low,close,volume,label
count,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000
mean,1.054341,1.054429,1.054251,1.054342,122.697988,1.054035
std,0.444148,0.444188,0.444107,0.444149,112.177245,0.444490
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.102145,1.102214,1.102102,1.102143,55.349948,1.102115
50%,1.173609,1.173676,1.173538,1.173607,123.118047,1.173607
75%,1.319153,1.319270,1.319081,1.319154,163.647447,1.319154
max,1.484259,1.484420,1.484097,1.484257,1089.065415,1.484257


In [377]:
df.shape

(3652, 6)

In [378]:
dff = df.dropna() # dropping all missing number and setting to a new variable because dff will be scaled in the next step

In [379]:
dff.isnull().sum() # no more missing number

open      0
high      0
low       0
close     0
volume    0
label     0
dtype: int64

In [380]:
dff.head()  # dff has been scaled

,open,high,low,close,volume,label
date,,,,,,
2010-01-01,1.433194,1.433288,1.433089,1.433178,60.243939,0.000000
2010-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,1.433178
2010-01-03,1.431891,1.432094,1.431711,1.431913,25.013333,0.000000
2010-01-04,1.436431,1.436693,1.436159,1.436451,55.569028,1.431913
2010-01-05,1.440870,1.441108,1.440588,1.440881,55.477153,1.436451


In [381]:
dff.columns

Index(['open', 'high', 'low', 'close', 'volume', 'label'], dtype='object')

In [382]:
tss = TimeSeriesSplit(n_splits=3)
X = dff[['open', 'high', 'low', 'close', 'volume']]
y = dff.label

In [383]:
# splitting data 80/20 between test and train test respectivly.

for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

X_train.shape, X_test.shape, y_train.shape, y_test.shape # checking the that features' and targets shape have similar length

((2739, 5), (913, 5), (2739,), (913,))

In [384]:
scaler = StandardScaler()

In [385]:
scaler.fit(X_train)

StandardScaler()

In [386]:
scaled_X_train = scaler.fit_transform(X_train)

In [387]:
scaled_X_train

array([[ 0.78730537,  0.78722309,  0.78737419,  0.78726707, -0.48646029],
       [-2.37067024, -2.37066619, -2.37067776, -2.37066931, -0.99416895],
       [ 0.78443493,  0.78459234,  0.78433791,  0.78447987, -0.7833679 ],
       ...,
       [ 0.14515969,  0.14512411,  0.14522194,  0.14516652,  0.44248982],
       [ 0.14578692,  0.14574066,  0.14583894,  0.14578192,  0.36437017],
       [-2.37067024, -2.37066619, -2.37067776, -2.37066931, -0.99416895]])

In [388]:
scaled_X_train = pd.DataFrame(scaled_X_train, columns= X_train.columns.values)

In [389]:
scaled_X_test = scaler.transform(X_test)

In [390]:
scaled_X_test = pd.DataFrame(scaled_X_test, columns= X_train.columns.values)

In [391]:
scaled_X_test

,open,high,low,close,volume
0,0.146001,0.145842,0.146181,0.146006,-0.501081
1,0.137333,0.137244,0.137426,0.137325,0.023185
2,0.131222,0.131120,0.131333,0.131216,-0.079264
3,0.129074,0.129000,0.129156,0.129068,0.217295
4,0.136718,0.136651,0.136813,0.136726,0.199657
...,...,...,...,...,...
908,0.084961,0.084850,0.085092,0.084969,-0.198813
909,-2.370670,-2.370666,-2.370678,-2.370669,-0.994169
910,0.092491,0.092379,0.092585,0.092474,-0.762483
911,0.097009,0.096893,0.097138,0.097009,-0.228288


In [392]:
scaled_X_train.shape, scaled_X_test.shape

((2739, 5), (913, 5))

In [393]:
scaled_linreg_model = LinearRegression() # creates the liner regression instance

In [394]:
scaled_linreg_model.fit(scaled_X_train, y_train) # fits the training inputs to the linear regressor

LinearRegression()

In [395]:
# we will check the model performace using .score()

In [396]:
print("The train r-square error is : {}".format(scaled_linreg_model.score(scaled_X_train,y_train).round(3))) 

The train r-square error is : 0.115


In [397]:
print("The test r-square error is : {}".format(scaled_linreg_model.score(scaled_X_test,y_test).round(3)))

The test r-square error is : 0.235


In [398]:
# Compare the R² errors for test and train and explain the outcome.
"""I iteratively remove the X values one after the other until the last one. 
My observation is that the model's performances plummeted from  (X and y scaled ) 0.802 and  0.791 respectively

The reduction in the model's performance is justified because, for models to perform optimally, 
they need data and lots of it. In other words, the more data we feed to them, the better and more efficient they are in 
learning about the data and the easier they are able to generalise accurately when confronted with an unseen data source."""



"I iteratively remove the X values one after the other until the last one. \nMy observation is that the model's performances plummeted from  (X and y scaled ) 0.802 and  0.791 respectively\n\nThe reduction in the model's performance is justified because, for models to perform optimally, \nthey need data and lots of it. In other words, the more data we feed to them, the better and more efficient they are in \nlearning about the data and the easier they are able to generalise accurately when confronted with an unseen data source."

In [399]:
predictions = scaled_linreg_model.predict(X_test) # predicting on the test data.

In [400]:
predictions

array([ 7.33641993, 14.12424857, 12.7968738 , 16.62419332, 16.37818678,
       17.32315346,  1.07588973,  6.4095945 , 13.16429642, 15.54436391,
       18.21526269, 16.58375508, 17.75854692,  1.07588973, 11.50971486,
       15.17732006, 20.14210815, 15.94424555, 21.41733674, 20.32082724,
        1.07588973,  7.84206073, 16.44899637, 17.78361539, 20.26007966,
       21.18056012, 21.32081927,  1.07588973, 12.61481707, 18.87054738,
       18.11113854, 18.6103249 , 17.71128549, 19.63113273,  1.07588973,
        9.53655322, 14.75892736, 16.81699264, 21.554689  , 18.11188329,
       19.14674271,  1.07588973,  6.48066363, 16.60818468, 17.28375263,
       20.03314166, 19.89487487, 20.17583104,  1.07588973,  5.21874511,
       14.71642599, 15.80173429, 16.0876127 , 17.82349069, 20.21037338,
        1.07588973,  9.97937628, 16.75938148, 22.23883709, 19.683603  ,
       20.14728181, 20.12619308,  1.07588973,  8.53016104, 14.82487575,
       20.08479196, 19.11336438, 21.19533821, 21.89078028,  1.07

In [401]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'label'], dtype='object')

In [402]:
# Larry William’s %R
def disparity_5(df):
     return 100 * df['close'] / df['close'].rolling(window=14).mean()

df['14_moving_avg'] = disparity_5(df)

In [403]:
df.dropna(inplace=True)

In [404]:
df.isnull().sum()

open             0
high             0
low              0
close            0
volume           0
label            0
14_moving_avg    0
dtype: int64

In [405]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'label', '14_moving_avg'], dtype='object')

In [406]:
X = df[['open', 'high', 'low', 'close', 'volume']] #drop all columns except close
y = df.label

In [407]:
# splitting data 80/20 between test and train test respectivly.
    
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

X_train.shape, X_test.shape, y_train.shape, y_test.shape
# checking the that features' and targets shape have similar length

((2730, 5), (909, 5), (2730,), (909,))

In [408]:
unscaled_linreg_model = LinearRegression()

In [409]:
unscaled_linreg_model.fit(X_train,y_train)

LinearRegression()

In [410]:
print("The train r-square error is : {}".format(unscaled_linreg_model.score(X_train,y_train).round(3))) 

The train r-square error is : 0.114


In [411]:
unscaled_predictions = unscaled_linreg_model.predict(X_test)

In [412]:
print("The train r-square error is : {}".format(unscaled_linreg_model.score(X_test, y_test).round(3))) 

The train r-square error is : 0.234


In [413]:
"""I performed Linear regression analysis on unscaled X and Y train test. 
The model performed poorly with scores of 0.115 and 0.235 on train and test sets respectively. 
This is likely due to the huge disparity between some values found within different columns."""

'I performed Linear regression analysis on unscaled X and Y train test. \nThe model performed poorly with scores of 0.115 and 0.235 on train and test sets respectively. \nThis is likely due to the huge disparity between some values found within different columns.'

In [414]:
df.head(2)

,open,high,low,close,volume,label,14_moving_avg
date,,,,,,,
2010-01-14,1.450969,1.451202,1.450697,1.450982,55.535208,1.450402,117.468300
2010-01-15,1.440126,1.440370,1.439829,1.440132,57.748636,1.450982,116.543034


In [415]:
ohlv_df = df.iloc[:, :6] # slicing all rows and first  6 columns
ohlv_df

,open,high,low,close,volume,label
date,,,,,,
2010-01-14,1.450969,1.451202,1.450697,1.450982,55.535208,1.450402
2010-01-15,1.440126,1.440370,1.439829,1.440132,57.748636,1.450982
2010-01-16,0.000000,0.000000,0.000000,0.000000,0.000000,1.440132
2010-01-17,1.436004,1.436192,1.435818,1.436037,33.940000,0.000000
2010-01-18,1.437623,1.437833,1.437363,1.437643,55.667292,1.436037
...,...,...,...,...,...,...
2019-12-27,1.114447,1.114498,1.114401,1.114452,94.375684,1.109451
2019-12-28,0.000000,0.000000,0.000000,0.000000,0.000000,1.114452
2019-12-29,1.117864,1.117915,1.117801,1.117858,27.491448,0.000000


In [416]:

ohlcv_new = ohlv_df[(ohlv_df["low"] != 0) & (ohlv_df["low"] != 0) & (ohlv_df["label"] != 0)] # filtering the df for where df is not zeros

In [417]:
(ohlcv_new == 0).sum() # assert statement shows no zeros

open      0
high      0
low       0
close     0
volume    0
label     0
dtype: int64

In [418]:
ohlcv_new

,open,high,low,close,volume,label
date,,,,,,
2010-01-14,1.450969,1.451202,1.450697,1.450982,55.535208,1.450402
2010-01-15,1.440126,1.440370,1.439829,1.440132,57.748636,1.450982
2010-01-18,1.437623,1.437833,1.437363,1.437643,55.667292,1.436037
2010-01-19,1.433307,1.433536,1.433012,1.433312,54.864653,1.437643
2010-01-20,1.415888,1.416130,1.415571,1.415886,55.764236,1.433312
...,...,...,...,...,...,...
2019-12-25,1.108536,1.108557,1.108512,1.108536,8.440950,1.108461
2019-12-26,1.109451,1.109475,1.109429,1.109451,36.627798,1.108536
2019-12-27,1.114447,1.114498,1.114401,1.114452,94.375684,1.109451


In [419]:

"""
​
  
%K= 100∗(CP−L14)/(H14−L14)
​
 
where:
C=Most recent closing price
L14=Low of the 14 previous trading sessions
H14=Highest price traded during the same 14-day period
​
"""
# Calculate an additional feature Stochastic slow %D

'\n\u200b\n  \n%K= 100∗(CP−L14)/(H14−L14)\n\u200b\n \nwhere:\nC=Most recent closing price\nL14=Low of the 14 previous trading sessions\nH14=Highest price traded during the same 14-day period\n\u200b\n'

In [420]:
df["L14"] = df.low.rolling(window=14).min() # Min of the 14-day mva (lowest low)
df["H14"] = df.high.rolling(window=14).max() # max of the  14-day mva (highest high)
df["%K_stochastic_slow_%K"] = (df["close"]-df["L14"])*100/(df["H14"]-df["L14"])
#%D is the 3-day moving average of %K
df["Stochastic_slow_%D"] = df["%K_stochastic_slow_%K"].rolling(3).mean()

""" %K stochastic slows shows how often the price close above a particular price in during a price range(moving average)

The indicator can be used for planning entry and exit especially when used with the %D stochastic which provide an additional test as for where the price is heading i.e up or down
As an example a %K of 50 means that, during a price range (MVAs), the price of an asset close above the current price 50% of times. 
According to investopedia, a %K of 80 and above is considered good enough to enter or stay in a position.
"""

' %K stochastic slows shows how often the price close above a particular price in during a price range(moving average)\n\nThe indicator can be used for planning entry and exit especially when used with the %D stochastic which provide an additional test as for where the price is heading i.e up or down\nAs an example a %K of 50 means that, during a price range (MVAs), the price of an asset close above the current price 50% of times. \nAccording to investopedia, a %K of 80 and above is considered good enough to enter or stay in a position.\n'

In [421]:
df.head(2)

,open,high,low,close,volume,label,14_moving_avg,L14,H14,%K_stochastic_slow_%K,Stochastic_slow_%D
date,,,,,,,,,,,
2010-01-14,1.450969,1.451202,1.450697,1.450982,55.535208,1.450402,117.468300,NaN,NaN,NaN,NaN
2010-01-15,1.440126,1.440370,1.439829,1.440132,57.748636,1.450982,116.543034,NaN,NaN,NaN,NaN


In [422]:


candlestick = go.Candlestick(
                            x=ohlcv_new.index,
                            open=ohlcv_new['open'],
                            high=ohlcv_new['high'],
                            low=ohlcv_new['low'],
                            close=ohlcv_new['close']
                            )

fig = go.Figure(data=[candlestick])
fig.update_layout(title = "EURUSD", xaxis_title="Date", yaxis_title="EURUSD price" , template="plotly_dark")

fig.show()

In [423]:
# Create a figure based on OHLC candles covering the test period (the 20% of data)

In [424]:

candlestick = go.Candlestick(
                            x=X_test.index,
                            open=X_test['open'],
                            high=X_test['high'],
                            low=X_test['low'],
                            close=X_test['close']
                            )

fig = go.Figure(data=[candlestick])
fig.update_layout(title = "EURUSD", xaxis_title="Date", yaxis_title="EURUSD price" , template="plotly_dark")

fig.show()

In [425]:
# Grade 3

In [426]:
# Calculate additional feature RSI (relative strength index)
# RS = Average Gain / Average Loss
"""            100
    RSI = 100 - --------
                 1 + RS"""

'            100\n    RSI = 100 - --------\n                 1 + RS'

In [432]:
change_in_closing_price = df["close"].diff()
df["avg_gain"]= -1*change_in_closing_price.clip(upper=0)
df["avg_loss"] = change_in_closing_price.clip(lower=0)

In [433]:
df

,open,high,low,close,volume,label,14_moving_avg,L14,H14,%K_stochastic_slow_%K,Stochastic_slow_%D,avg_gain,avg_loss
date,,,,,,,,,,,,,
2010-01-14,1.450969,1.451202,1.450697,1.450982,55.535208,1.450402,117.468300,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,1.440126,1.440370,1.439829,1.440132,57.748636,1.450982,116.543034,NaN,NaN,NaN,NaN,0.010850,0.000000
2010-01-16,0.000000,0.000000,0.000000,0.000000,0.000000,1.440132,0.000000,NaN,NaN,NaN,NaN,1.440132,0.000000
2010-01-17,1.436004,1.436192,1.435818,1.436037,33.940000,0.000000,116.183938,NaN,NaN,NaN,NaN,-0.000000,1.436037
2010-01-18,1.437623,1.437833,1.437363,1.437643,55.667292,1.436037,116.305871,NaN,NaN,NaN,NaN,-0.000000,0.001606
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,1.114447,1.114498,1.114401,1.114452,94.375684,1.109451,117.013194,0.0,1.114942,99.955976,99.577653,-0.000000,0.005000
2019-12-28,0.000000,0.000000,0.000000,0.000000,0.000000,1.114452,0.000000,0.0,1.114942,0.000000,66.461819,1.114452,0.000000
2019-12-29,1.117864,1.117915,1.117801,1.117858,27.491448,0.000000,117.324536,0.0,1.117915,99.994816,66.650264,-0.000000,1.117858
